# OPI Quick Start - Jupyter Notebook

This notebook demonstrates basic OPI simulation using synthetic data.

## 1. Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '..')  # Add parent directory to path

import opi
from opi.io import xy2lonlat
from opi.viz import plot_topography_map, plot_precipitation_map, plot_isotope_map

print("OPI version:", opi.__version__)
print("All imports successful!")

## 2. Create Synthetic Topography

In [ ]:
# Create a Gaussian mountain
dem = opi.create_synthetic_dem(
    topo_type='gaussian',
    grid_size=(500e3, 500e3),    # 500km x 500km domain
    grid_spacing=(2000, 2000),   # 2km resolution
    lon0=0,                      # Center longitude
    lat0=45,                     # Center latitude (45°N)
    amplitude=2000,              # 2000m peak height
    sigma=(50e3, 50e3),          # Gaussian width
    output_file='../examples/data/gaussian_topo.mat'
)

print(f"Topography shape: {dem['hGrid'].shape}")
print(f"Height range: {dem['hGrid'].min():.1f} - {dem['hGrid'].max():.1f} m")

### Visualize Topography

In [ ]:
# Plot topography using Haxby colormap (oceanographic style)
cmap = opi.haxby()

fig, ax = plot_topography_map(
    dem['lon'], dem['lat'], dem['hGrid'],
    title='Gaussian Mountain (2000m peak)',
    cmap=cmap
)
plt.show()

## 3. Run OPI Simulation

In [ ]:
# Method 1: Direct calculation without run file
from opi.calc_one_wind import calc_one_wind

# Grid vectors
x, y = dem['x'], dem['y']
h_grid = dem['hGrid']

# Parameters (9 parameters)
beta = np.array([
    10.0,    # U: Wind speed (m/s)
    90.0,    # azimuth: Wind direction (degrees from north)
    290.0,   # T0: Sea-level temperature (K)
    0.25,    # M: Mountain height number
    0.0,     # kappa: Eddy diffusivity (m²/s)
    1000.0,  # tau_c: Condensation time (s)
    -5e-3,   # d2h0: Base d2H (fraction, -50 permil)
    -2e-3,   # d_d2h0_d_lat: d2H latitudinal gradient (fraction/degree)
    0.7      # f_p0: Residual precipitation fraction
])

# Physical constants
f_c = 1e-4    # Coriolis parameter at 45°N (~1e-4 rad/s)
h_r = 540     # Isotope exchange distance (m)

print("Running OPI simulation...")
print(f"Wind: {beta[0]:.1f} m/s from {beta[1]:.1f}°")
print(f"Temperature: {beta[2]:.1f} K")

In [ ]:
# Run simulation (no sample data - just forward calculation)
# Create dummy sample data for function signature
n_samples = 5
sample_x = np.linspace(-100000, 100000, n_samples)
sample_y = np.zeros(n_samples)
sample_d2h = np.array([-100, -110, -120, -130, -140]) * 1e-3
sample_d18o = sample_d2h / 8  # Approximate MWL

# Calculate catchment indices (simplified)
from opi.catchment import catchment_indices
ij_catch = [[(i, 125)] for i in range(n_samples)]  # Simplified
ptr_catch = list(range(n_samples + 1))

result = calc_one_wind(
    beta=beta,
    f_c=f_c,
    h_r=h_r,
    x=x,
    y=y,
    lat=np.array([45.0]),
    lat0=45.0,
    h_grid=h_grid,
    b_mwl_sample=np.array([9.47e-3, 8.03]),  # Meteoric water line
    ij_catch=ij_catch,
    ptr_catch=ptr_catch,
    sample_d2h=sample_d2h,
    sample_d18o=sample_d18o,
    cov=np.array([[1e-6, 0], [0, 1e-6]]),
    n_parameters_free=9,
    is_fit=False  # Forward calculation only
)

# Unpack results
(chi_r2, nu, std_residuals, z_bar, T, gamma_env, gamma_sat, gamma_ratio,
 rho_s0, h_s, rho0, h_rho, d18o0, d_d18o0_d_lat, tau_f,
 p_grid, f_m_grid, r_h_grid, evap_d2h_grid, u_evap_d2h_grid,
 evap_d18o_grid, u_evap_d18o_grid, d2h_grid, d18o_grid,
 i_wet, d2h_pred, d18o_pred) = result

print(f"\nSimulation complete!")
print(f"Chi-square: {chi_r2:.4f}")
print(f"Degrees of freedom: {nu}")
print(f"Precipitation rate: {p_grid.min()*1000*86400:.2f} - {p_grid.max()*1000*86400:.2f} mm/day")
print(f"d2H range: {d2h_grid.min()*1000:.1f} to {d2h_grid.max()*1000:.1f} permil")

## 4. Visualize Results

In [ ]:
# Plot precipitation
fig, ax = plot_precipitation_map(
    dem['lon'], dem['lat'], p_grid,
    title=f'Precipitation Rate (U={beta[0]:.1f} m/s, Az={beta[1]:.0f}°)',
    cmap='Blues'
)
plt.show()

In [ ]:
# Plot isotopes
fig, axes = plot_isotope_map(
    dem['lon'], dem['lat'], d2h_grid, d18o_grid,
    title_prefix='Predicted',
    cmap='RdYlBu_r'
)
plt.show()

## 5. Cross-Section Analysis

In [ ]:
from opi.viz import plot_cross_section

# Plot cross-section through center
fig, axes = plot_cross_section(
    x, y, h_grid, d2h_grid,
    section_y=0,
    variable_name='d2H (permil)',
    cmap='RdYlBu_r'
)
plt.show()

## 6. Save Results

In [ ]:
# Save to MATLAB format
from opi.io import save_opi_results

results_dict = {
    'lon': dem['lon'],
    'lat': dem['lat'],
    'h_grid': h_grid,
    'beta': beta,
    'chi_r2': chi_r2,
    'nu': nu,
    'p_grid': p_grid,
    'd2h_grid': d2h_grid,
    'd18o_grid': d18o_grid,
    'f_m_grid': f_m_grid,
    'r_h_grid': r_h_grid,
}

save_opi_results('../examples/data/opi_results.mat', results_dict)
print("Results saved to opi_results.mat")

## Next Steps

- Try different wind speeds and directions
- Experiment with different topography types
- See `02_parameter_fitting.ipynb` for optimization
- See `03_real_data.ipynb` for working with real datasets